In [60]:
# Convert into three csv for songs, presentors and genres, indexed by song_id

import os
import csv
import string

data_path = "data"
output_path = "csv"
files = [os.path.join(data_path, f) for f in os.listdir(data_path)]
        
    
class Song:
    def __init__(self, name, artist, lyric, presentor, genres):
        self.name = name
        self.artist = artist
        self.lyric = lyric
        self.presentors = presentor
        self.genres = genres


song_csv = open(os.path.join(output_path, 'songs.csv'), 'w')
performer_csv = open(os.path.join(output_path, 'performers.csv'), 'w')
genre_csv = open(os.path.join(output_path, 'genres.csv'), 'w')

song_headers = ['song_id', 'name', 'artist', 'lyric']
song_writer = csv.DictWriter(song_csv, song_headers)
song_writer.writeheader()

performer_headers = ['song_id', 'performer']
performer_writer = csv.DictWriter(performer_csv, performer_headers)
performer_writer.writeheader()

genre_headers = ['song_id', 'genre']
genre_writer = csv.DictWriter(genre_csv, genre_headers)
genre_writer.writeheader()

def strip_punctuations(s):
    return  ' '.join(word.strip(string.punctuation) for word in s.split())

def zipdict(headers, values):
    return dict(zip(headers, values))

for p in files:
    with open(p) as f:
        try:
            song_id =  int(os.path.splitext(os.path.basename(p).replace('/.txt', ''))[0])
            lines = f.readlines()
            lines = [line.strip() for line in lines]
            stripped_lyrics = strip_punctuations('.'.join(lines[4:]))
            song = Song(lines[0], lines[1], stripped_lyrics, lines[2].split(','), lines[3].split(','))
            song_writer.writerow(zipdict(song_headers, [song_id, song.name, song.artist, song.lyric]))

            for performer in song.presentors:
                performer_writer.writerow(zipdict(performer_headers, [song_id, performer]))

            for genre in song.genres:
                genre_writer.writerow(zipdict(genre_headers, [song_id, genre]))
        except:
            continue
            
song_csv.close()
performer_csv.close()
genre_csv.close()


In [70]:
from pyspark import SparkConf, SparkContext, SQLContext
import pyspark.sql.functions as func

spark = pyspark.sql.SparkSession.builder\
    .master("local") \
    .appName('app')\
    .getOrCreate()
sc = spark.sparkContext

In [82]:
#help(spark.read.csv)

songs = spark.read.csv('file:///Users/louis/VietSongAnalysis/csv/songs.csv', header=True, sep=",")
performers = spark.read.csv('file:///Users/louis/VietSongAnalysis/csv/performers.csv', header=True, sep=",")
genres = spark.read.csv('file:///Users/louis/VietSongAnalysis/csv/performers.csv', header=True, sep=",")

11846